#  🤗 Transformers

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* https://huggingface.co/docs/transformers/index
* https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.pipeline.task
* https://huggingface.co/docs/transformers/preprocessing
* https://huggingface.co/blog/getting-started-with-embeddings
* https://huggingface.co/sentence-transformers
* https://habr.com/ru/articles/704592/

## Задачи для совместного разбора

1\. Обсудите основные возможности и экосистему пакета 🤗 Transformers на примере задачи поиска ответа на вопрос в тексте.

In [1]:
text = """The seminars on Deep Learning and Natural Language Processing were truly captivating,
providing a deep dive into the intricacies of these disciplines.
The wealth of knowledge and insights gained during the sessions was commendable.
However, it's disheartening to note the scarcity of homework assignments.
Anastasia, in particular, is quite concerned that the limited number of assignments might
fall short of even reaching 30. While the seminars were intellectually stimulating,
the desire for more hands-on practice through assignments remains strong,
as it is crucial for reinforcing the theoretical understanding acquired during the classes."""

In [2]:
question1 = "What would be the ideal number of homework assignments for Anastasia"
question2 = "What are the shortcomings of the course?"

In [3]:
import transformers

In [4]:
from transformers import pipeline

question_answerer = pipeline(
    "question-answering",
    model='distilbert-base-uncased-distilled-squad'
)

result = question_answerer(
    question=question2,
    context=text
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
result

{'score': 0.2378229945898056,
 'start': 272,
 'end': 304,
 'answer': 'scarcity of homework assignments'}

In [6]:
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
import torch

tokenizer = DistilBertTokenizer.from_pretrained(
    'distilbert-base-uncased-distilled-squad'
)

model = DistilBertForQuestionAnswering.from_pretrained(
    'distilbert-base-uncased-distilled-squad'
)

question = question1

inputs = tokenizer(question, text, return_tensors="pt")
inputs

{'input_ids': tensor([[  101,  2054,  2052,  2022,  1996,  7812,  2193,  1997, 19453, 14799,
          2005, 19447,   102,  1996, 17239,  2006,  2784,  4083,  1998,  3019,
          2653,  6364,  2020,  5621, 14408, 17441,  1010,  4346,  1037,  2784,
         11529,  2046,  1996, 20014, 14735,  9243,  1997,  2122, 12736,  1012,
          1996,  7177,  1997,  3716,  1998, 20062,  4227,  2076,  1996,  6521,
          2001,  4012,  3549, 20782,  1012,  2174,  1010,  2009,  1005,  1055,
          9841, 14644,  6528,  2075,  2000,  3602,  1996, 11228, 12972,  1997,
         19453, 14799,  1012, 19447,  1010,  1999,  3327,  1010,  2003,  3243,
          4986,  2008,  1996,  3132,  2193,  1997, 14799,  2453,  2991,  2460,
          1997,  2130,  4285,  2382,  1012,  2096,  1996, 17239,  2020,  7789,
          2135, 27295,  1010,  1996,  4792,  2005,  2062,  2398,  1011,  2006,
          3218,  2083, 14799,  3464,  2844,  1010,  2004,  2009,  2003, 10232,
          2005, 27788, 29278,  6129,  

In [7]:
with torch.no_grad():
    outputs = model(**inputs)

In [8]:
outputs

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[-3.9710, -2.4511, -4.7110, -5.4935, -4.4091, -4.1409, -5.7816, -7.0359,
         -4.6655, -5.3889, -5.9580, -3.3720, -4.5949, -3.9335, -2.2237, -5.6928,
         -3.9662, -4.5620, -6.3194, -4.4003, -4.2180, -4.2241, -6.1591, -5.3823,
         -4.9748, -5.0480, -5.6633, -4.9366, -5.5484, -4.8528, -5.0129, -6.6144,
         -6.4383, -5.7038, -6.4587, -6.1019, -7.3245, -6.2075, -4.6985, -5.4507,
         -4.7950, -4.5115, -7.0736, -4.1096, -7.1219, -4.6241, -5.9617, -5.9044,
         -5.5952, -3.7005, -6.9322, -5.0653, -7.0248, -5.8431, -5.2738, -5.2059,
         -6.1076, -4.2529, -6.6603, -6.4773, -4.6573, -6.6450, -6.2248, -6.1602,
         -5.6475, -4.7885, -3.8283, -2.2721, -4.8317, -6.1482, -1.0441, -3.5488,
         -4.0351, -0.6817, -5.2630, -3.5373, -4.5299, -4.6355, -2.7195, -2.0185,
         -1.4997, -1.1423,  2.3250,  1.3184, -1.8264, -2.7621, -0.2712,  0.7548,
          0.9143, -0.1617, -2.7816,  2.9039,  2.8023,  6

In [9]:
answer_start_index = torch.argmax(outputs.start_logits)
answer_end_index = torch.argmax(outputs.end_logits)

predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)

'30'

## Задачи для самостоятельного решения

<p class="task" id="1"></p>

1\. Среди предобученных моделей найдите модель для перевода текста с русского языка на английский. Протестируйте данную модель на нескольких предложениях, используя `transformers.pipeline`. Выведите результаты работы в следующем виде:

```
sentence1_ru -> sentence1_en
sentence2_ru -> sentence2_en
```

Получите перевод для всех текстов из файла `RuBQ_2.0_test.json` и посчитайте BLEU-score.

- [ ] Проверено на семинаре

In [10]:
import pandas as pd

test = pd.read_json('/content/drive/MyDrive/пм21_финашка/3 курс/NLP/04_rnn/RuBQ_2.0_test.json')

In [11]:
# test_ru = ' '.join(test.question_text.values)
test_ru = test.question_text.values
test_ru

array(['Какой стране принадлежит знаменитый остров Пасхи?',
       'С какой музыкальной группой неразрывно связано имя Мика Джаггера?',
       'Где находится Летний сад?',
       'Какой город является столицей Туркмении?',
       'В каком городе издавалась с 1857 г. А. Герценом и Н. Огаревым первая российская революционная газета "Колокол"?',
       'В какой стране находится второй из самых высоких действующих вулканов с забавным названием Попокатепетль (5452 м)?',
       'В каком виде спорта прославилась Курникова?',
       'В каком городе родился Вольфганг Амадей Моцарт?',
       'В каком городе убили Джона Леннона?',
       'Гражданином какой страны был изобретатель телеграфного кода Морзе?',
       'Как называлась столица Крымского ханства?',
       'Как назывался корабль Хана Соло?',
       'Какой музыкальный инструмент прославил имя Генри Стейнве?',
       'Какой стране принадлежит авиакомпания "Malev"?',
       'Кто автор поэмы "Энеида"?',
       'Кто автор романа "Колыбель для 

In [12]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ru-en")
# model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ru-en")


In [13]:
!pip install  sentencepiece

In [14]:
import sentencepiece

Translation = pipeline(
    "translation",
    model='Helsinki-NLP/opus-mt-ru-en'
)

source.spm:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/803k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [15]:
len(test_ru)

580

In [16]:
sentence1_en = Translation(test_ru[50])[0]['translation_text']
sentence2_en = Translation(test_ru[90])[0]['translation_text']
print(f'{test_ru[50]},\n {sentence1_en}')
print(f'\n{test_ru[90]},\n {sentence2_en}')

В какой прыжковой дисциплине выступал олимпийский чемпион Виктор Санеев?,
 What kind of jump discipline was the Olympic champion Victor Saneeev in?

Кто написал роман «Триумфальная арка»?,
 Who wrote the novel "The Triumph Arch"?


In [17]:
translation_en = []

for i in test_ru:
  sentence_en = Translation(i)[0]['translation_text']
  translation_en.append(sentence_en)
  print(f'\n{i},\n {sentence_en}')


Какой стране принадлежит знаменитый остров Пасхи?,
 Which country does the famous Easter Island belong to?

С какой музыкальной группой неразрывно связано имя Мика Джаггера?,
 What kind of music group is Mick Jagger's name inextricably linked?

Где находится Летний сад?,
 Where is the Summer Garden?

Какой город является столицей Туркмении?,
 What city is the capital of Turkmenistan?

В каком городе издавалась с 1857 г. А. Герценом и Н. Огаревым первая российская революционная газета "Колокол"?,
 In which city was A. Gerzen and N. Ogarev's first Russian revolutionary newspaper, the Bell, published since 1857?

В какой стране находится второй из самых высоких действующих вулканов с забавным названием Попокатепетль (5452 м)?,
 Which country is the second highest active volcano with the funny name Popokatepetl (5,452 m)?

В каком виде спорта прославилась Курникова?,
 What kind of sport is Kurnikov famous for?

В каком городе родился Вольфганг Амадей Моцарт?,
 What city was Wolfgang Amada

KeyboardInterrupt: 

In [18]:
translation_en

['Which country does the famous Easter Island belong to?',
 "What kind of music group is Mick Jagger's name inextricably linked?",
 'Where is the Summer Garden?',
 'What city is the capital of Turkmenistan?',
 "In which city was A. Gerzen and N. Ogarev's first Russian revolutionary newspaper, the Bell, published since 1857?",
 'Which country is the second highest active volcano with the funny name Popokatepetl (5,452 m)?',
 'What kind of sport is Kurnikov famous for?',
 'What city was Wolfgang Amaday Mozart born in?',
 'In which city was John Lennon murdered?',
 "Which country was the inventor of Morse's telegraph code?",
 'What was the name of the capital of Crimean sanity?',
 "What was the name of Han Solo's ship?",
 'What musical instrument has the name Henry Steinweh famous for?',
 'Which country does Malev own?',
 "Who's the writer of Eneida's poem?",
 'Who is the author of the novel "Cat Cradle"?',
 'Who wrote the novel "The Winter of Concern"?',
 'Who was the first to discover S

In [19]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [20]:
test_en = test.question_eng.values
test_en[:10]

array(['Which country does the famous Easter island belong to?',
       "Which music group is Mick Jagger's name inextricably linked to?",
       'Where is the Summer garden?',
       'Which city is the capital of Turkmenistan?',
       'In which city was the first Russian revolutionary newspaper Kolokol published since 1857 by A. Herzen and N. Ogarev?',
       'Which country has the second highest active volcano with the funny name Popocatepetl (5452 m)?',
       'In what sport was Kournikova famous?',
       'What city was Wolfgang Amadeus Mozart born in?',
       'What city was John Lennon killed in?',
       'What country was the inventor of the Morse code Telegraph a citizen of?'],
      dtype=object)

In [22]:
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu

print('Bleu score =', corpus_bleu([[i] for i in test_en][:10], translation_en[:10]))

Bleu score = 0.8292133325231998


<p class="task" id="2"></p>

2\. Загрузите данные из файла `reviews_polarity.csv`. Среди предобученных моделей найдите модель для классификации тональности русскоязычного текста (позитивный/негативный или позитивный/негативный/нейтральный). Протестируйте данную модель на нескольких предложениях, используя `transformers.pipeline`. Выведите результаты работы в следующем виде:

```
sentence1 -> class1
sentence2 -> class2
...
```

Получите прогноз для всех текстов из файла и посчитайте F1-score. Для ускорения работы модели вы можете перенести ее на GPU и использовать возможности pipeline для работы с батчами и генераторами.

- [ ] Проверено на семинаре

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/пм21_финашка/3 курс/NLP/05_transformers/reviews_polarity.csv')
df.head()

,text,class
0,"Очень хорошо что открылась 5 ка, теперь не над...",1
1,"Тесно, вечная сутолока, между рядами трудно ра...",0
2,Магазин в пешей доступности. После ремонта и р...,1
3,Магазин хороший цены и скидки нормальные токо ...,1
4,Сложно найти в торговом центре. А магазин - норм),1


In [ ]:
text = df.text.values
text[:30]

array(['Очень хорошо что открылась 5 ка, теперь не надо далеко ехать все рядом!',
       'Тесно, вечная сутолока, между рядами трудно разойтись, грязно. Была пару раз. Больше не пойду. ',
       'Магазин в пешей доступности. После ремонта и реконструкции стал намного лучше и уютнее. Радует появившаяся пекарня и выпечка.',
       'Магазин хороший цены и скидки нормальные токо все время у вас работает одна касса тк не у всех есть деньги на картах всегда',
       'Сложно найти в торговом центре. А магазин - норм)',
       'После ремонта магазин в нутри стал ещё лучше. Больше ассортимент товаров, плюс скидки на продукцию. Вежливый персонал. Есть небольшая парковка.',
       'Все свежее, продукты.',
       'Наша любимая 🤩 \nСделали перестановку, выпечку добавили, кофе с собой можно организовать! Улучшение всегда хорошо! 👌🏻',
       'Большой выбор продуктов, разнообразные акции',
       'Неплохой магазин, но Спар, который там был раньше, был гораздо лучше',
       'Достойный супермаркет этой

In [ ]:
text

array(['Очень хорошо что открылась 5 ка, теперь не надо далеко ехать все рядом!',
       'Тесно, вечная сутолока, между рядами трудно разойтись, грязно. Была пару раз. Больше не пойду. ',
       'Магазин в пешей доступности. После ремонта и реконструкции стал намного лучше и уютнее. Радует появившаяся пекарня и выпечка.',
       ...,
       'Очень хочется пожелать этому магазину стать таким же уютным, как на Савушкина 115/5',
       'Нравится ваш магазин, персонал одекватный, порядок.',
       'Не очень  персонал'], dtype=object)

In [ ]:
import numpy as np
np.unique(df['class'].values)

array([0, 1])

In [ ]:
!pip install fugashi

In [ ]:
!pip install unidic_lite

In [ ]:
import fugashi
import unidic_lite

In [ ]:
classifier = pipeline("sentiment-analysis", model="jarvisx17/japanese-sentiment-analysis", device=0, batched=True)

In [ ]:
classifier(text[0])[0]['label']

Keyword arguments {'batched': True} not recognized.


'positive'

In [ ]:
text

array(['Очень хорошо что открылась 5 ка, теперь не надо далеко ехать все рядом!',
       'Тесно, вечная сутолока, между рядами трудно разойтись, грязно. Была пару раз. Больше не пойду. ',
       'Магазин в пешей доступности. После ремонта и реконструкции стал намного лучше и уютнее. Радует появившаяся пекарня и выпечка.',
       ...,
       'Очень хочется пожелать этому магазину стать таким же уютным, как на Савушкина 115/5',
       'Нравится ваш магазин, персонал одекватный, порядок.',
       'Не очень  персонал'], dtype=object)

In [ ]:
sentiment = []

for i in text[:1000]:
  if classifier(i)[0]['label'] == 'positive':
    sentiment.append(1)
  elif classifier(i)[0]['label'] == 'negative':
    sentiment.append(0)

  print(i, '\nemotion -->', classifier(i)[0]['label'])

Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.


Очень хорошо что открылась 5 ка, теперь не надо далеко ехать все рядом! 
emotion --> positive
Тесно, вечная сутолока, между рядами трудно разойтись, грязно. Была пару раз. Больше не пойду.  
emotion --> negative
Магазин в пешей доступности. После ремонта и реконструкции стал намного лучше и уютнее. Радует появившаяся пекарня и выпечка. 
emotion --> negative
Магазин хороший цены и скидки нормальные токо все время у вас работает одна касса тк не у всех есть деньги на картах всегда 
emotion --> positive
Сложно найти в торговом центре. А магазин - норм) 
emotion --> negative


Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.


После ремонта магазин в нутри стал ещё лучше. Больше ассортимент товаров, плюс скидки на продукцию. Вежливый персонал. Есть небольшая парковка. 
emotion --> negative
Все свежее, продукты. 
emotion --> positive
Наша любимая 🤩 
Сделали перестановку, выпечку добавили, кофе с собой можно организовать! Улучшение всегда хорошо! 👌🏻 
emotion --> positive
Большой выбор продуктов, разнообразные акции 
emotion --> positive
Неплохой магазин, но Спар, который там был раньше, был гораздо лучше 
emotion --> negative
Достойный супермаркет этой сети. 
emotion --> negative


Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.


...

Парковка и правда удобная,удобный магазин 
emotion --> negative
Смотрите внимательно на ценники, отдал за обычный молочный коктейль 43 вместо заявленных 26 на ценнике 
emotion --> positive
"Пятерочка" на Двоечку 
emotion --> negative
Сегодня наблюдал картину, как струдник магазина обыскивал пенсинерку, после того как ничего не нашли, даже не извинились, позорище, примите меры!!!  
emotion --> positive


Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.


Практически не бывает очередей , а в остальном обычный магазин. 
emotion --> negative
Пятёрочка как пятёрочка))) этим и хороша!👍 
emotion --> positive
Отличный ассортимент, всегда есть то, что нужно, отлично работают кассиры,всегда вежливые и доброжелательные, прекрасная атмосфера, спасибо 
emotion --> negative
Мне очень удобно ходить с коляской в этот магазин. В остальном обычный супермаркет 
emotion --> negative


Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.
Keyword arguments {'batched': True} not recognized.


Даже ценники на месте. Но не удобный внутренний переход 
emotion --> negative
Удобный подъезд к магазину, персонал приветлив, однако качество товаров и своевренность замены ценников на прилавках, оставляет желать лучшего 
emotion --> negative
Много лишних магазинчиков внутри.  Центр притяжения алкашни 
emotion --> positive
Супер, кассиры вежливые 
emotion --> positive
Цены подскочили 
emotion --> positive


In [ ]:
from sklearn.metrics import f1_score

f1_score(df['class'][:len(sentiment)], sentiment)

0.5579886455798865

<p class="task" id="3"></p>

3\. Создайте токенизатор, соответствующий модели из предыдущего задания, используя класс `AutoTokenizer`. Возьмите одно предложение из набора данных и токенизируйте его, используя созданный объект. Выведите на экран полученный результат. Выполните обратное преобразование: получите текст по набору токенов и выведите результат на экран.

Получите батч из 10 предложений и токенизируйте его. Продемонстрируйте возможности токенизатора для паддинга, обрезки, преобразования в тензоры.

- [ ] Проверено на семинаре

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("jarvisx17/japanese-sentiment-analysis",  batched=True)

In [ ]:
token_res = tokenizer(text[0], return_tensors="pt")
token_res

{'input_ids': tensor([[    2,   351,  6789, 30661,  6966,   386, 25621,  6373,  7950,  6373,
           388,  6229,  6373,   379,  6229,  6233,  6375,  7466, 32626,  6232,
          6966,    35,   375,  6230,    26,   383,  6374,  7368, 26186,  6966,
           378,  6374,   378,  6230,  6857,  6373,   369,  6230,  6533,  6374,
          6233,  6373,   370,  6945,  6230,  6229,  6966,   367,  6232,  6374,
           381,  6824,  6857,  6373,  6858,    15,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
token_res['input_ids'][0]

tensor([    2,   351,  6789, 30661,  6966,   386, 25621,  6373,  7950,  6373,
          388,  6229,  6373,   379,  6229,  6233,  6375,  7466, 32626,  6232,
         6966,    35,   375,  6230,    26,   383,  6374,  7368, 26186,  6966,
          378,  6374,   378,  6230,  6857,  6373,   369,  6230,  6533,  6374,
         6233,  6373,   370,  6945,  6230,  6229,  6966,   367,  6232,  6374,
          381,  6824,  6857,  6373,  6858,    15,     3])

In [ ]:
tokenizer.decode(token_res['input_ids'][0])

'[CLS] Очень хорошо что открылась 5 ка, теперь не надо далеко ехать все рядом! [SEP]'

In [ ]:
batch = text[:10]
batch

array(['Очень хорошо что открылась 5 ка, теперь не надо далеко ехать все рядом!',
       'Тесно, вечная сутолока, между рядами трудно разойтись, грязно. Была пару раз. Больше не пойду. ',
       'Магазин в пешей доступности. После ремонта и реконструкции стал намного лучше и уютнее. Радует появившаяся пекарня и выпечка.',
       'Магазин хороший цены и скидки нормальные токо все время у вас работает одна касса тк не у всех есть деньги на картах всегда',
       'Сложно найти в торговом центре. А магазин - норм)',
       'После ремонта магазин в нутри стал ещё лучше. Больше ассортимент товаров, плюс скидки на продукцию. Вежливый персонал. Есть небольшая парковка.',
       'Все свежее, продукты.',
       'Наша любимая 🤩 \nСделали перестановку, выпечку добавили, кофе с собой можно организовать! Улучшение всегда хорошо! 👌🏻',
       'Большой выбор продуктов, разнообразные акции',
       'Неплохой магазин, но Спар, который там был раньше, был гораздо лучше'],
      dtype=object)

In [ ]:
token_batch = tokenizer(batch.tolist())
token_batch

In [ ]:
# паддинг
tokenizer(batch.tolist(), return_tensors='pt', padding=True)

In [ ]:
# обрезка
tokenizer(batch.tolist(), return_tensors='pt',  truncation=True, max_length=8)

{'input_ids': tensor([[    2,   351,  6789, 30661,  6966,   386, 25621,     3],
        [    2,   355,  6374,  6232,  6355,  6373,    26,     3],
        [    2,   349,  6230,  7707,  6230,  6559, 31024,     3],
        [    2,   349,  6230,  7707,  6230,  6559, 31024,     3],
        [    2,   354,  6533,  6373,  8003,  6355,  6373,     3],
        [    2,   352,  6373,  6232,  6533,  6374,   381,     3],
        [    2,   339,  6232,  6374,   382,  6376,  6374,     3],
        [    2,   350,  6230,  7950,  6230,   376,  7948,     3],
        [    2,   338, 28887,  6966,  7950,  6373,  6231,     3],
        [    2,   350,  6374,  7368,  6533,  6373,  6945,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],


<p class="task" id="4"></p>

4\. Решите задачу 2, создав объект токенизатора (`AutoTokenizer`) и модель (`AutoModelForSequenceClassification`).

- [ ] Проверено на семинаре

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("jarvisx17/japanese-sentiment-analysis",  batched=True)
model = AutoModelForSequenceClassification.from_pretrained("jarvisx17/japanese-sentiment-analysis")

In [ ]:
input = tokenizer(text[0],  return_tensors="pt")
input

{'input_ids': tensor([[    2,   351,  6789, 30661,  6966,   386, 25621,  6373,  7950,  6373,
           388,  6229,  6373,   379,  6229,  6233,  6375,  7466, 32626,  6232,
          6966,    35,   375,  6230,    26,   383,  6374,  7368, 26186,  6966,
           378,  6374,   378,  6230,  6857,  6373,   369,  6230,  6533,  6374,
          6233,  6373,   370,  6945,  6230,  6229,  6966,   367,  6232,  6374,
           381,  6824,  6857,  6373,  6858,    15,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
with torch.no_grad():
    outputs = model(**inputs)

In [ ]:
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-1.6613,  2.1322]]), hidden_states=None, attentions=None)

In [ ]:
sentiment2 = []

for i in range(len(text[:1000])):
    inputs = tokenizer(text[:1000][i], return_tensors="pt")
    outputs = model(**inputs)
    # print(outputs.logits)
    pred = th.argmax(outputs.logits).item()
    sentiment2.append(pred)
    if (i+1)% 100 == 0:
      print(text[:1000][i],'class -->', pred)


Появился магазин и жизнь в районе изменилась к лучшему. class --> 1
Хороший чистый магазин. class --> 0
Удачное расположение рядом с домом большая площадь магазина есть парковка широкий ассортимент товаров) class --> 0
Не удобное расположение внутри магазина,чтоб что то найти надо либо знать уже где или звать работника магазина.Не знаю как для других по мне так не удобно! class --> 0
Нормальный магазин. 
 class --> 1
Кассы самообслуживания (часто не срабатывает услуга карты магазина о начислении баллов),не высокий ассортимент фруктов и овощей. class --> 0
У магазина, для меня, единственный минус - жуткие лабиринты - в остальном - чисто, выбор нормальный, приятные продавцы class --> 0
Странные сотрудники иногда  class --> 0
Хороший ассортимент, просторно, замечательное сочетание цены и качества class --> 1
Цены подскочили class --> 1


In [ ]:
f1_score(df['class'][:len(sentiment2)], sentiment2)

0.5579886455798865

<p class="task" id="5"></p>

5\. Используя пакет `sentence-transformers`, загрузите модель `distiluse-base-multilingual-cased-v1`. Используя эту модель, получите эмбеддинги всех предложений из набора данных. Во время генерации эмбеддингов используйте батчи размера 64 и отобразите процесс получения эмбеддингов в виде полосы загрузки.

Уменьшите размерность эмбеддингов при помощи PCA и отобразите точки в двумерном пространстве, раскрасив их в цвет соответствующего им класса.

- [ ] Проверено на семинаре

In [ ]:
pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("DataikuNLP/distiluse-base-multilingual-cased-v1")

.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.57k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/452 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

In [ ]:
from tqdm import tqdm

## Обратная связь
- [ ] Хочу получить обратную связь по решению

Оценка вероятности в случае триграмм:
$$\hat{P}(w_k|w_{k−2}w_{k−1}) = \begin{cases}
& P(w_k|w_{k−2}w_{k−1}) \text{ , } F(w_{k−2}w_{k−1}w_k)>0 \\
& \alpha(w_{k−2}^{k-1})\hat{P}(w_k|w_{k−1}) \text{ , } F(w_{k−2}w_{k−1}w_k)=0
\end{cases}
$$
* Коэффициент $\alpha$ необходим для корректного распределения остаточной вероятности n-грамм
в соответствии с распределением вероятности (n-1)-грамм.
* Если не вводить $\alpha$, оценка будет ошибочной, т.к. не будет выполняться равенство:
$$\sum_{i,j} P(w_k|w_i w_j) = 1$